In [114]:
from pyspark.sql import SparkSession
import pandas as pd
import scipy as sc
from itertools import permutations
from scipy.sparse import coo_matrix
import numpy as np
from google.cloud import storage


In [125]:

client = storage.Client()
bucket = client.bucket('bgse-datawarehousing-random-tweets')
paths = []
for blob in bucket.list_blobs(prefix=''):
    paths += ['gs://bgse-datawarehousing-random-tweets/'+blob.name]



In [126]:
path = 'gs://bgse-datawarehousing-random-tweets/2019-02-26T00:00:30.657Z'
#raw_data = sc.textFile(path)
data = spark.read.json(paths[0:5])

In [127]:
data.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: bo

In [128]:
a=data.rdd \
    .filter(lambda l: l.entities is not None) \
    .filter(lambda x: len(x.entities.hashtags) > 1) \
    .map(lambda x: [i.text.lower() for i in x.entities.hashtags]) \
    .flatMap(lambda r: permutations(r,2)) \
    .map(lambda q: (q,1)) \
    .reduceByKey(lambda a,b:a+b ) \
    .collect()
a

[((u'buildthewall', u'drainthedeepstate'), 1),
 ((u'voted', u'california'), 1),
 ((u'thewalliscoming', u'buildthewallandcrimewillfall'), 1),
 ((u'bebest', u'stophate'), 1),
 ((u'brexit', u'newsnight'), 12),
 ((u'brexiteers', u'brexitshambles'), 5),
 ((u'draftdodger', u'coward'), 2),
 ((u'wuwei', u'feed'), 1),
 ((u'politicalcartoon', u'uk'), 1),
 ((u'fucking', u'coward'), 1),
 ((u'brexit', u'brexitvote'), 1),
 ((u'estad\xe3olix\xe3oesquerdalha',
   u'euvimdegra\xe7aporumbrasilnovoporumnovobrasil'),
  1),
 ((u'draintheswamp', u'americafirst'), 1),
 ((u'protesting', u'trump'), 1),
 ((u'referendum', u'peoplesvote'), 1),
 ((u'breadlinebernie', u'socialism'), 3),
 ((u'prettyjeongyeon', u'jeongyeon'), 1),
 ((u'qanon', u'trump2020'), 2),
 ((u'trump', u'heslying'), 2),
 ((u'foxnews', u'rnc'), 1),
 ((u'putinspuppet', u'trumpcolluded'), 2),
 ((u'chamatodos', u'aprovadossusepe'), 1),
 ((u'buildthewallandcrimewillfall', u'promisesmadepromiseskept'), 1),
 ((u'1hour', u'living'), 1),
 ((u'lechuguinos

In [129]:
col_list=data.rdd \
    .filter(lambda l: l.entities is not None) \
    .filter(lambda x: len(x.entities.hashtags) > 1) \
    .flatMap(lambda x: [i.text.lower() for i in x.entities.hashtags]) \
    .distinct()\
    .collect()
len(col_list)

1069

In [130]:
sort_list = sorted(col_list)
dict_of_words = {sort_list[i] : i for i in range(0,len(sort_list))}
dict_of_words

{u'tribunals': 911,
 u'liar': 519,
 u'\u30cb\u30e5\u30fc\u30b9\u3067\u82f1\u5358\u8a9e': 1059,
 u'supportsmallstreamers': 844,
 u'tomorrowspaperstoday': 894,
 u'global': 368,
 u'resistance': 746,
 u'drumpf': 242,
 u'obstruction': 639,
 u'hatinc': 391,
 u'demonrats': 207,
 u'gb': 352,
 u'politicslive': 681,
 u'flashback': 321,
 u'conservative': 165,
 u'jussiesmollett': 472,
 u'trumpholiday': 923,
 u'c4news': 127,
 u'eligible': 256,
 u'demonicrats': 206,
 u'bonespurs': 88,
 u'breadlinebernie': 97,
 u'cnnsitroom': 156,
 u'march16th': 554,
 u'the_collapse_of_america_is_near': 870,
 u'unfittoserve': 970,
 u'whitesupremacy': 1026,
 u'voted': 1005,
 u'ivankatrump': 453,
 u'yellowvestprotests': 1042,
 u'amlo': 30,
 u'women': 1034,
 u'tb': 858,
 u'fakenews': 294,
 u'mayday': 564,
 u'p2': 656,
 u'vet': 992,
 u'minnesnowta': 579,
 u'promisesmadepromiseskept': 697,
 u'owleyes': 655,
 u'vegan': 986,
 u'rednationrising': 730,
 u'teamgodvek': 861,
 u'donaldtrumpcartoon': 227,
 u'dutchpm': 246,
 u'tar

In [131]:
row  = np.array([dict_of_words[i[0][0]] for i in a])
col  = np.array([dict_of_words[i[0][1]] for i in a])
data = np.array([i[1] for i in a])
B = coo_matrix((data, (row, col)), shape=(len(sort_list), len(sort_list)))

In [132]:
B

<1069x1069 sparse matrix of type '<type 'numpy.int64'>'
	with 5568 stored elements in COOrdinate format>